In [ ]:
"""
HCA-DR (Hierarchical Cell-Aware DrugReflector) 数据预处理 - 板级匹配修复版
================================================================================

致命错误修复：使用板级匹配（Plate-level Matching）而非全局平均

核心改变：
1. 为每个样本匹配其所在 plate 的 DMSO 对照
2. 如果样本所在 plate 没有 DMSO，使用同批次或同细胞系的最近 plate
3. 保留 plate-level 的批次信息，避免引入全局平均的噪声
================================================================================
"""

import numpy as np
import pandas as pd
import h5py
from pathlib import Path
import pickle
from scipy import stats
from scipy.stats import rankdata
from typing import Dict, Tuple, Optional, List
import warnings
import gc
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# ==================== 配置路径 ====================
LINCS_DATA_DIR = Path("D:/科研/Models/drugreflector/datasets/LINCS2020")
PROCESSED_DATA_DIR = Path("D:/科研/Models/drugreflector/processed_data")
OUTPUT_DIR = Path("D:/科研/Models/drugreflector/processed_data")
VIZ_DIR = Path("D:/科研/Models/drugreflector/visualizations/hca_dr")

# 创建输出目录
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)
VIZ_DIR.mkdir(exist_ok=True, parents=True)

# 输入文件
TRAINING_DATA_FILE = "training_data_lincs2020_chemfiltered_1201_l.pkl"
LEVEL3_CTL_FILE = "level3_beta_ctl_n188708x12328.gctx"
GENE_INFO_FILE = "geneinfo_beta.txt"
INST_INFO_FILE = "instinfo_beta.txt"

# 输出文件
OUTPUT_FILE = "hca_dr_training_data_plate_matched.pkl"

print("=" * 80)
print("🔧 HCA-DR Data Preprocessing - PLATE-LEVEL MATCHING VERSION")
print("=" * 80)
print(f"\n📁 Configuration:")
print(f"   LINCS data: {LINCS_DATA_DIR}")
print(f"   Processed data: {PROCESSED_DATA_DIR}")
print(f"   Output: {OUTPUT_DIR / OUTPUT_FILE}\n")


# ==================== 1. 加载基因信息 ====================
def load_gene_info(gene_file: Path) -> Tuple[pd.DataFrame, np.ndarray]:
    """加载基因信息并获取landmark基因索引"""
    print("📖 Loading gene information...")
    gene_info = pd.read_csv(gene_file, sep='\t')
    
    print(f"   ✓ Total genes: {len(gene_info):,}")
    landmark_mask = gene_info['feature_space'] == 'landmark'
    landmark_indices = np.where(landmark_mask.values)[0]
    print(f"   ✓ Landmark genes: {len(landmark_indices)}")
    
    return gene_info, landmark_indices


# ==================== 2. 加载训练数据 ====================
def load_training_data(training_path: Path) -> Dict:
    """加载已处理的训练数据"""
    print("\n📖 Loading processed training data...")
    
    with open(training_path, 'rb') as f:
        training_data = pickle.load(f)
    
    meta = training_data['sample_meta']
    print(f"   ✓ Samples: {len(training_data['X']):,}")
    print(f"   ✓ Compounds: {len(training_data['compound_names']):,}")
    print(f"   ✓ Genes: {training_data['X'].shape[1]}")
    
    # 确保有 plate 信息
    plate_col = 'det_plate' if 'det_plate' in meta.columns else 'rna_plate'
    print(f"   ✓ Using '{plate_col}' as plate identifier")
    print(f"   ✓ Unique plates in training: {meta[plate_col].nunique()}")
    
    return training_data


# ==================== 3. 加载 Level 3 DMSO 数据 ====================
def load_level3_controls(level3_file: Path, 
                         inst_file: Path,
                         landmark_indices: np.ndarray) -> Tuple[np.ndarray, pd.DataFrame]:
    """加载Level 3 DMSO对照数据"""
    print("\n📖 Loading Level 3 DMSO control data...")
    
    # 加载实例信息
    level3_meta_all = pd.read_csv(inst_file, sep='\t', low_memory=False)
    print(f"   ✓ Total Level 3 instances: {len(level3_meta_all):,}")
    
    # 检查可能的 control 标识列
    if 'pert_type' in level3_meta_all.columns:
        control_mask = level3_meta_all['pert_type'].isin(['ctl_vehicle', 'control'])
        print(f"   ✓ Filtering by pert_type: {control_mask.sum():,} control samples")
    elif 'pert_iname' in level3_meta_all.columns:
        control_mask = level3_meta_all['pert_iname'].str.contains('DMSO|control', case=False, na=False)
        print(f"   ✓ Filtering by pert_iname: {control_mask.sum():,} control samples")
    else:
        # 如果没有明确标识，假设文件名已经告诉我们这是 control 数据
        # 使用数据矩阵的列数来确定
        print(f"   ⚠️  No control identifier found, assuming all samples in data matrix are controls")
        control_mask = None
    
    # 加载数据
    print("   Loading expression matrix (this may take a while)...")
    with h5py.File(level3_file, 'r') as f:
        # 只加载landmark基因
        data = f['0/DATA/0/matrix'][:]
        data_landmark = data[landmark_indices, :]
        print(f"   ✓ Data shape: {data_landmark.shape}")
    
    # 根据数据矩阵的大小来匹配 metadata
    n_samples_in_data = data_landmark.shape[1]
    
    if control_mask is not None:
        level3_meta = level3_meta_all[control_mask].reset_index(drop=True)
        
        # 如果过滤后仍不匹配，取前 n_samples_in_data 行
        if len(level3_meta) != n_samples_in_data:
            print(f"   ⚠️  Metadata count ({len(level3_meta)}) != data count ({n_samples_in_data})")
            print(f"   Taking first {n_samples_in_data} rows from filtered metadata")
            level3_meta = level3_meta.iloc[:n_samples_in_data].reset_index(drop=True)
    else:
        # 直接取前 n_samples_in_data 行
        level3_meta = level3_meta_all.iloc[:n_samples_in_data].reset_index(drop=True)
    
    # 验证
    assert len(level3_meta) == data_landmark.shape[1], \
        f"Metadata和数据维度不匹配: {len(level3_meta)} vs {data_landmark.shape[1]}"
    
    print(f"   ✓ Loaded {data_landmark.shape[1]:,} control samples")
    print(f"   ✓ Genes: {data_landmark.shape[0]}")
    print(f"   ✓ Metadata matched successfully")
    
    return data_landmark.T, level3_meta


# ==================== 4. 计算每个 Plate 的 DMSO 均值 ====================
def compute_plate_dmso_means(level3_data: np.ndarray,
                            level3_meta: pd.DataFrame) -> Dict[str, np.ndarray]:
    """
    计算每个plate的DMSO对照均值
    
    关键：这是批次效应控制的核心 - 保持plate-level的信息
    """
    print("\n" + "=" * 80)
    print("🧪 Computing Plate-wise DMSO Control Means")
    print("=" * 80)
    
    plate_col = 'det_plate' if 'det_plate' in level3_meta.columns else 'rna_plate'
    print(f"   Using '{plate_col}' as plate identifier")
    
    unique_plates = level3_meta[plate_col].unique()
    print(f"   ✓ Total unique plates: {len(unique_plates):,}")
    
    plate_means = {}
    plate_sample_counts = []
    
    for plate in tqdm(unique_plates, desc="   Computing plate means"):
        plate_mask = level3_meta[plate_col] == plate
        plate_data = level3_data[plate_mask]
        
        if len(plate_data) > 0:
            plate_mean = np.mean(plate_data, axis=0).astype(np.float32)
            plate_means[plate] = plate_mean
            plate_sample_counts.append(len(plate_data))
    
    print(f"\n   ✓ Computed means for {len(plate_means):,} plates")
    print(f"   📊 Samples per plate: mean={np.mean(plate_sample_counts):.1f}, "
          f"median={np.median(plate_sample_counts):.0f}, "
          f"min={np.min(plate_sample_counts):.0f}, "
          f"max={np.max(plate_sample_counts):.0f}")
    
    return plate_means


# ==================== 5. 构建 Plate -> Cell 映射 ====================
def build_plate_to_cell_mapping(level3_meta: pd.DataFrame) -> Dict[str, List[str]]:
    """构建plate到细胞系的映射，用于fallback策略"""
    print("\n📋 Building plate-to-cell mapping...")
    
    plate_col = 'det_plate' if 'det_plate' in level3_meta.columns else 'rna_plate'
    cell_col = 'cell_iname' if 'cell_iname' in level3_meta.columns else 'cell_mfc_name'
    
    plate_to_cells = defaultdict(set)
    
    for _, row in level3_meta.iterrows():
        plate = row[plate_col]
        cell = row[cell_col]
        if pd.notna(plate) and pd.notna(cell):
            plate_to_cells[plate].add(cell)
    
    # 转换为list
    plate_to_cells = {k: list(v) for k, v in plate_to_cells.items()}
    
    print(f"   ✓ Built mapping for {len(plate_to_cells)} plates")
    
    return plate_to_cells


# ==================== 6. 🔥 核心修复：板级匹配 ====================
def match_plate_level_contexts(training_meta: pd.DataFrame,
                               plate_dmso_means: Dict[str, np.ndarray],
                               plate_to_cells: Dict[str, List[str]],
                               n_genes: int = 978) -> Tuple[np.ndarray, np.ndarray, Dict]:
    """
    🔥 CRITICAL FIX: Plate-level context matching
    
    为每个训练样本匹配其所在plate的DMSO对照
    
    策略：
    1. 优先：使用样本所在plate的DMSO均值（最准确）
    2. Fallback 1：如果plate没有DMSO，使用同细胞系其他plates的均值
    3. Fallback 2：如果细胞系没有任何DMSO，使用零向量（会被过滤）
    
    参数：
        training_meta: 训练数据的元信息
        plate_dmso_means: 每个plate的DMSO均值 {plate_id: mean_vector}
        plate_to_cells: plate到细胞系的映射
        n_genes: 基因数量
    
    返回：
        X_ctx: 上下文矩阵 (n_samples, n_genes)
        matching_quality: 匹配质量标记 (n_samples,)
            0: 完美匹配（same plate）
            1: Fallback（same cell, different plate）
            2: Missing（无匹配）
        matching_stats: 统计信息
    """
    print("\n" + "=" * 80)
    print("🔥 PLATE-LEVEL CONTEXT MATCHING (Critical Fix)")
    print("=" * 80)
    
    n_samples = len(training_meta)
    
    # 确定列名
    plate_col = 'det_plate' if 'det_plate' in training_meta.columns else 'rna_plate'
    cell_col = 'cell_iname' if 'cell_iname' in training_meta.columns else 'cell_mfc_name'
    
    print(f"   Matching strategy:")
    print(f"   1. Same plate DMSO (best)")
    print(f"   2. Same cell, different plates (fallback)")
    print(f"   3. Zero vector (missing)")
    print(f"\n   Processing {n_samples:,} samples...")
    
    # 初始化
    X_ctx = np.zeros((n_samples, n_genes), dtype=np.float32)
    matching_quality = np.zeros(n_samples, dtype=np.int8)
    
    # 构建细胞系到其所有plates的映射（用于fallback）
    cell_to_plates = defaultdict(list)
    for plate, cells in plate_to_cells.items():
        for cell in cells:
            if plate in plate_dmso_means:  # 只添加有DMSO数据的plate
                cell_to_plates[cell].append(plate)
    
    # 统计变量
    n_exact_match = 0
    n_fallback = 0
    n_missing = 0
    
    # 逐样本匹配
    for idx in tqdm(range(n_samples), desc="   Matching contexts"):
        row = training_meta.iloc[idx]
        sample_plate = row[plate_col]
        sample_cell = row[cell_col]
        
        # 策略1: 使用同plate的DMSO（最佳）
        if sample_plate in plate_dmso_means:
            X_ctx[idx] = plate_dmso_means[sample_plate]
            matching_quality[idx] = 0
            n_exact_match += 1
        
        # 策略2: Fallback - 使用同细胞系其他plates的平均
        elif sample_cell in cell_to_plates and len(cell_to_plates[sample_cell]) > 0:
            cell_plates = cell_to_plates[sample_cell]
            cell_plate_means = [plate_dmso_means[p] for p in cell_plates]
            X_ctx[idx] = np.mean(cell_plate_means, axis=0)
            matching_quality[idx] = 1
            n_fallback += 1
        
        # 策略3: 无匹配 - 零向量
        else:
            X_ctx[idx] = np.zeros(n_genes, dtype=np.float32)
            matching_quality[idx] = 2
            n_missing += 1
    
    # 统计信息
    print(f"\n   📊 Matching Results:")
    print(f"   ✅ Exact match (same plate): {n_exact_match:,} ({n_exact_match/n_samples*100:.1f}%)")
    print(f"   🔄 Fallback (same cell, diff plate): {n_fallback:,} ({n_fallback/n_samples*100:.1f}%)")
    print(f"   ❌ Missing: {n_missing:,} ({n_missing/n_samples*100:.1f}%)")
    
    matching_stats = {
        'n_exact_match': n_exact_match,
        'n_fallback': n_fallback,
        'n_missing': n_missing,
        'pct_exact': n_exact_match / n_samples * 100,
        'pct_fallback': n_fallback / n_samples * 100,
        'pct_missing': n_missing / n_samples * 100
    }
    
    return X_ctx, matching_quality, matching_stats


# ==================== 7. 应用 Rank-based INT 归一化 ====================
def apply_rank_based_int_per_sample(X_ctx: np.ndarray,
                                    matching_quality: np.ndarray) -> np.ndarray:
    """
    对每个基因应用Rank-based Inverse Normal Transformation
    
    重要：只在有效样本（matching_quality < 2）上计算排名
    """
    print("\n" + "=" * 80)
    print("📊 Applying Rank-based INT Normalization")
    print("=" * 80)
    
    n_samples, n_genes = X_ctx.shape
    
    # 只在有效样本上进行归一化
    valid_mask = matching_quality < 2
    n_valid = valid_mask.sum()
    
    print(f"   Valid samples for normalization: {n_valid:,} / {n_samples:,}")
    
    X_ctx_int = np.zeros_like(X_ctx)
    
    for g in tqdm(range(n_genes), desc="   Normalizing genes"):
        gene_values = X_ctx[valid_mask, g]
        
        if len(gene_values) > 1:
            # 计算排名
            ranks = rankdata(gene_values, method='average')
            
            # 转换为标准正态分位数
            quantiles = ranks / (len(gene_values) + 1)
            int_values = stats.norm.ppf(quantiles)
            
            # 填充到完整矩阵
            X_ctx_int[valid_mask, g] = int_values
    
    # 验证归一化
    valid_data = X_ctx_int[valid_mask]
    print(f"\n   📊 INT Validation (valid samples only):")
    print(f"      • Mean per gene: {valid_data.mean(axis=0).mean():.6f}")
    print(f"      • Std per gene: {valid_data.std(axis=0).mean():.6f}")
    print(f"      • Overall mean: {valid_data.mean():.6f}")
    print(f"      • Overall std: {valid_data.std():.6f}")
    
    return X_ctx_int


# ==================== 8. 构建细胞系 ID 映射 ====================
def build_cell_id_mapping(training_meta: pd.DataFrame,
                         matching_quality: np.ndarray) -> Dict[str, int]:
    """构建细胞系ID映射"""
    print("\n" + "=" * 80)
    print("🏷️  Building Cell Line ID Mapping")
    print("=" * 80)
    
    cell_col = 'cell_iname' if 'cell_iname' in training_meta.columns else 'cell_mfc_name'
    
    # 只考虑有有效匹配的细胞系
    valid_mask = matching_quality < 2
    valid_cells = training_meta[valid_mask][cell_col].unique()
    
    # 构建映射
    sorted_cells = sorted(list(valid_cells))
    cell_id_to_idx = {cell: idx for idx, cell in enumerate(sorted_cells)}
    
    print(f"   ✓ Built mapping for {len(cell_id_to_idx)} cell lines")
    
    return cell_id_to_idx


# ==================== 9. 分配细胞系 ID ====================
def assign_cell_ids(training_meta: pd.DataFrame,
                   cell_id_to_idx: Dict[str, int]) -> np.ndarray:
    """为每个样本分配细胞系ID"""
    print("\n📋 Assigning cell line IDs...")
    
    cell_col = 'cell_iname' if 'cell_iname' in training_meta.columns else 'cell_mfc_name'
    
    cell_ids = np.array([
        cell_id_to_idx.get(cell, -1) 
        for cell in training_meta[cell_col]
    ], dtype=np.int32)
    
    print(f"   ✓ Assigned IDs for {(cell_ids >= 0).sum():,} samples")
    
    return cell_ids


# ==================== 10. 构建最终数据集 ====================
def build_final_dataset(training_data: Dict,
                       X_ctx: np.ndarray,
                       matching_quality: np.ndarray,
                       cell_ids: np.ndarray,
                       cell_id_to_idx: Dict[str, int],
                       matching_stats: Dict,
                       filter_missing: bool = True) -> Dict:
    """构建最终的HCA-DR数据集"""
    print("\n" + "=" * 80)
    print("🚀 Building Final HCA-DR Dataset")
    print("=" * 80)
    
    if filter_missing:
        # 过滤掉没有有效匹配的样本
        valid_mask = matching_quality < 2
        n_before = len(matching_quality)
        n_after = valid_mask.sum()
        
        print(f"   Filtering samples without valid context...")
        print(f"   Before: {n_before:,}")
        print(f"   After: {n_after:,}")
        print(f"   Removed: {n_before - n_after:,}")
        
        # 过滤数据
        X_pert = training_data['X'][valid_mask]
        X_ctx_filtered = X_ctx[valid_mask]
        cell_ids_filtered = cell_ids[valid_mask]
        matching_quality_filtered = matching_quality[valid_mask]
        folds = training_data['folds'][valid_mask]
        sample_meta = training_data['sample_meta'][valid_mask].reset_index(drop=True)
        
        # 重建化合物标签
        unique_perts = sorted(sample_meta['pert_id'].unique())
        new_pert_to_idx = {pert: idx for idx, pert in enumerate(unique_perts)}
        y = np.array([new_pert_to_idx[p] for p in sample_meta['pert_id']], dtype=np.int32)
        
        compound_names = unique_perts
        pert_to_idx = new_pert_to_idx
    else:
        X_pert = training_data['X']
        X_ctx_filtered = X_ctx
        cell_ids_filtered = cell_ids
        matching_quality_filtered = matching_quality
        folds = training_data['folds']
        sample_meta = training_data['sample_meta']
        y = training_data['y']
        compound_names = training_data['compound_names']
        pert_to_idx = training_data['pert_to_idx']
    
    print(f"\n   ✓ X_pert shape: {X_pert.shape}")
    print(f"   ✓ X_ctx shape: {X_ctx_filtered.shape}")
    print(f"   ✓ y shape: {y.shape}")
    print(f"   ✓ cell_ids shape: {cell_ids_filtered.shape}")
    
    # 构建数据字典
    hca_dr_data = {
        # 核心数据
        'X_pert': X_pert,
        'X_ctx': X_ctx_filtered,
        'y': y,
        'cell_ids': cell_ids_filtered,
        'matching_quality': matching_quality_filtered,
        'folds': folds,
        
        # 元数据
        'sample_meta': sample_meta,
        'gene_names': training_data['gene_names'],
        'compound_names': compound_names,
        'pert_to_idx': pert_to_idx,
        'cell_id_to_idx': cell_id_to_idx,
        'idx_to_cell_id': {v: k for k, v in cell_id_to_idx.items()},
        
        # 统计信息
        'n_samples': len(X_pert),
        'n_compounds': len(compound_names),
        'n_genes': X_pert.shape[1],
        'n_cell_lines': len(cell_id_to_idx),
        
        # 匹配质量统计
        'matching_stats': matching_stats,
        
        # 版本信息
        'version': 'plate_matched',
        'description': 'HCA-DR data with plate-level context matching (batch effect corrected)'
    }
    
    return hca_dr_data


# ==================== 11. 可视化分析 ====================
def visualize_matching_quality(hca_dr_data: Dict, output_dir: Path):
    """可视化匹配质量和批次效应控制"""
    print("\n📊 Creating visualization...")
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. 匹配质量分布
    ax1 = axes[0, 0]
    quality_labels = ['Exact\n(same plate)', 'Fallback\n(same cell)', 'Missing']
    quality_counts = [
        (hca_dr_data['matching_quality'] == 0).sum(),
        (hca_dr_data['matching_quality'] == 1).sum(),
        (hca_dr_data['matching_quality'] == 2).sum()
    ]
    colors = ['#2ecc71', '#f39c12', '#e74c3c']
    bars = ax1.bar(quality_labels, quality_counts, color=colors, alpha=0.7)
    ax1.set_ylabel('Number of Samples')
    ax1.set_title('Context Matching Quality', fontweight='bold')
    for bar, val in zip(bars, quality_counts):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2, height,
                f'{val:,}\n({val/len(hca_dr_data["matching_quality"])*100:.1f}%)',
                ha='center', va='bottom')
    
    # 2. X_ctx 分布
    ax2 = axes[0, 1]
    sample_ctx = hca_dr_data['X_ctx'][np.random.choice(len(hca_dr_data['X_ctx']), 
                                                         min(10000, len(hca_dr_data['X_ctx'])), 
                                                         replace=False)].flatten()
    ax2.hist(sample_ctx, bins=100, alpha=0.7, edgecolor='black', color='steelblue')
    ax2.set_xlabel('X_ctx Value')
    ax2.set_ylabel('Frequency')
    ax2.set_title('X_ctx Distribution (after INT)', fontweight='bold')
    ax2.axvline(0, color='red', linestyle='--', linewidth=2, label='Mean=0')
    ax2.legend()
    
    # 3. 每个细胞系的样本数
    ax3 = axes[0, 2]
    cell_counts = pd.Series(hca_dr_data['cell_ids']).value_counts().sort_values(ascending=True)
    top_cells = cell_counts.tail(15)
    cell_names = [hca_dr_data['idx_to_cell_id'][cid] for cid in top_cells.index]
    ax3.barh(range(len(top_cells)), top_cells.values, color='coral')
    ax3.set_yticks(range(len(top_cells)))
    ax3.set_yticklabels(cell_names, fontsize=9)
    ax3.set_xlabel('Number of Samples')
    ax3.set_title('Top 15 Cell Lines by Sample Count', fontweight='bold')
    
    # 4. X_pert vs X_ctx 相关性
    ax4 = axes[1, 0]
    sample_idx = np.random.choice(len(hca_dr_data['X_pert']), 
                                  min(5000, len(hca_dr_data['X_pert'])), 
                                  replace=False)
    pert_sample = hca_dr_data['X_pert'][sample_idx].flatten()
    ctx_sample = hca_dr_data['X_ctx'][sample_idx].flatten()
    ax4.hexbin(pert_sample, ctx_sample, gridsize=50, cmap='YlOrRd', mincnt=1)
    ax4.set_xlabel('X_pert Value')
    ax4.set_ylabel('X_ctx Value')
    ax4.set_title('X_pert vs X_ctx Correlation', fontweight='bold')
    plt.colorbar(ax4.collections[0], ax=ax4, label='Count')
    
    # 5. 匹配质量 pie chart
    ax5 = axes[1, 1]
    stats = hca_dr_data['matching_stats']
    sizes = [stats['n_exact_match'], stats['n_fallback']]
    labels = [f"Exact Match\n{stats['pct_exact']:.1f}%", 
              f"Fallback\n{stats['pct_fallback']:.1f}%"]
    ax5.pie(sizes, labels=labels, colors=['#2ecc71', '#f39c12'], 
            autopct='%d', startangle=90)
    ax5.set_title('Matching Strategy Distribution', fontweight='bold')
    
    # 6. Cross-validation fold 分布
    ax6 = axes[1, 2]
    fold_counts = pd.Series(hca_dr_data['folds']).value_counts().sort_index()
    bars = ax6.bar(fold_counts.index, fold_counts.values, 
                   color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax6.set_xlabel('Fold')
    ax6.set_ylabel('Number of Samples')
    ax6.set_title('Cross-Validation Fold Distribution', fontweight='bold')
    for bar, val in zip(bars, fold_counts.values):
        ax6.text(bar.get_x() + bar.get_width()/2, val, 
                f'{val:,}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig(output_dir / 'hca_dr_plate_matched_summary.png', 
                dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"   ✓ Saved visualization to: {output_dir / 'hca_dr_plate_matched_summary.png'}")


# ==================== 12. 主流程 ====================
def main():
    """主流程"""
    print("\n" + "=" * 80)
    print("🚀 STARTING HCA-DR DATA PREPARATION (PLATE-MATCHED VERSION)")
    print("=" * 80)
    
    # 1. 加载基因信息
    gene_info, landmark_indices = load_gene_info(LINCS_DATA_DIR / GENE_INFO_FILE)
    
    # 2. 加载训练数据
    training_data = load_training_data(PROCESSED_DATA_DIR / TRAINING_DATA_FILE)
    
    # 3. 加载 Level 3 DMSO 数据
    level3_data, level3_meta = load_level3_controls(
        LINCS_DATA_DIR / LEVEL3_CTL_FILE,
        LINCS_DATA_DIR / INST_INFO_FILE,
        landmark_indices
    )
    
    # 4. 计算每个 Plate 的 DMSO 均值
    plate_dmso_means = compute_plate_dmso_means(level3_data, level3_meta)
    
    # 5. 构建 Plate -> Cell 映射
    plate_to_cells = build_plate_to_cell_mapping(level3_meta)
    
    # 6. 🔥 板级匹配（核心修复）
    X_ctx_raw, matching_quality, matching_stats = match_plate_level_contexts(
        training_data['sample_meta'],
        plate_dmso_means,
        plate_to_cells,
        n_genes=978
    )
    
    # 7. 应用 INT 归一化
    X_ctx_int = apply_rank_based_int_per_sample(X_ctx_raw, matching_quality)
    
    # 8. 构建细胞系 ID 映射
    cell_id_to_idx = build_cell_id_mapping(
        training_data['sample_meta'], 
        matching_quality
    )
    
    # 9. 分配细胞系 ID
    cell_ids = assign_cell_ids(training_data['sample_meta'], cell_id_to_idx)
    
    # 10. 构建最终数据集
    hca_dr_data = build_final_dataset(
        training_data,
        X_ctx_int,
        matching_quality,
        cell_ids,
        cell_id_to_idx,
        matching_stats,
        filter_missing=True
    )
    
    # 11. 保存数据
    output_path = OUTPUT_DIR / OUTPUT_FILE
    print(f"\n💾 Saving HCA-DR dataset...")
    with open(output_path, 'wb') as f:
        pickle.dump(hca_dr_data, f, protocol=pickle.HIGHEST_PROTOCOL)
    
    file_size_mb = output_path.stat().st_size / (1024 * 1024)
    print(f"   ✓ Saved to: {output_path}")
    print(f"   ✓ File size: {file_size_mb:.2f} MB")
    
    # 12. 可视化
    visualize_matching_quality(hca_dr_data, VIZ_DIR)
    
    # 13. 最终总结
    print("\n" + "=" * 80)
    print("✅ HCA-DR DATA PREPROCESSING COMPLETE (PLATE-MATCHED)")
    print("=" * 80)
    print(f"\n📊 Final Dataset Summary:")
    print(f"   • Total samples: {hca_dr_data['n_samples']:,}")
    print(f"   • Compounds: {hca_dr_data['n_compounds']:,}")
    print(f"   • Cell lines: {hca_dr_data['n_cell_lines']}")
    print(f"   • Genes: {hca_dr_data['n_genes']}")
    print(f"\n📋 Matching Quality:")
    print(f"   • Exact match (same plate): {matching_stats['pct_exact']:.1f}%")
    print(f"   • Fallback (same cell): {matching_stats['pct_fallback']:.1f}%")
    print(f"   • Missing (filtered out): {matching_stats['pct_missing']:.1f}%")
    print(f"\n📁 Output Files:")
    print(f"   • Data: {output_path}")
    print(f"   • Visualization: {VIZ_DIR / 'hca_dr_plate_matched_summary.png'}")
    print(f"\n🎯 Ready for HCA-DR model training with proper batch effect control!")
    
    return hca_dr_data


if __name__ == "__main__":
    hca_dr_data = main()

🔧 HCA-DR Data Preprocessing - PLATE-LEVEL MATCHING VERSION

📁 Configuration:
   LINCS data: D:\科研\Models\drugreflector\datasets\LINCS2020
   Processed data: D:\科研\Models\drugreflector\processed_data
   Output: D:\科研\Models\drugreflector\processed_data\hca_dr_training_data_plate_matched.pkl


🚀 STARTING HCA-DR DATA PREPARATION (PLATE-MATCHED VERSION)
📖 Loading gene information...
   ✓ Total genes: 12,328
   ✓ Landmark genes: 978

📖 Loading processed training data...
   ✓ Samples: 509,006
   ✓ Compounds: 10,107
   ✓ Genes: 978
   ✓ Using 'det_plate' as plate identifier
   ✓ Unique plates in training: 4756

📖 Loading Level 3 DMSO control data...
   ✓ Total Level 3 instances: 3,026,460
   ✓ Filtering by pert_type: 102,696 control samples
   Loading expression matrix (this may take a while)...
   ✓ Data shape: (978, 12328)
   ⚠️  Metadata count (102696) != data count (12328)
   Taking first 12328 rows from filtered metadata
   ✓ Loaded 12,328 control samples
   ✓ Genes: 978
   ✓ Metadata ma

   Computing plate means: 100%|██████████| 4919/4919 [00:01<00:00, 2777.96it/s]



   ✓ Computed means for 4,919 plates
   📊 Samples per plate: mean=2.5, median=2, min=1, max=27

📋 Building plate-to-cell mapping...
   ✓ Built mapping for 4919 plates

🔥 PLATE-LEVEL CONTEXT MATCHING (Critical Fix)
   Matching strategy:
   1. Same plate DMSO (best)
   2. Same cell, different plates (fallback)
   3. Zero vector (missing)

   Processing 509,006 samples...


   Matching contexts: 100%|██████████| 509006/509006 [00:42<00:00, 11857.97it/s]



   📊 Matching Results:
   ✅ Exact match (same plate): 431,324 (84.7%)
   🔄 Fallback (same cell, diff plate): 77,682 (15.3%)
   ❌ Missing: 0 (0.0%)

📊 Applying Rank-based INT Normalization
   Valid samples for normalization: 509,006 / 509,006


   Normalizing genes: 100%|██████████| 978/978 [00:41<00:00, 23.36it/s]



   📊 INT Validation (valid samples only):
      • Mean per gene: -0.000029
      • Std per gene: 0.999511
      • Overall mean: -0.000029
      • Overall std: 0.999597

🏷️  Building Cell Line ID Mapping
   ✓ Built mapping for 57 cell lines

📋 Assigning cell line IDs...
   ✓ Assigned IDs for 509,006 samples

🚀 Building Final HCA-DR Dataset
   Filtering samples without valid context...
   Before: 509,006
   After: 509,006
   Removed: 0

   ✓ X_pert shape: (509006, 978)
   ✓ X_ctx shape: (509006, 978)
   ✓ y shape: (509006,)
   ✓ cell_ids shape: (509006,)

💾 Saving HCA-DR dataset...
   ✓ Saved to: D:\科研\Models\drugreflector\processed_data\hca_dr_training_data_plate_matched.pkl
   ✓ File size: 3943.06 MB

📊 Creating visualization...
   ✓ Saved visualization to: D:\科研\Models\drugreflector\visualizations\hca_dr\hca_dr_plate_matched_summary.png

✅ HCA-DR DATA PREPROCESSING COMPLETE (PLATE-MATCHED)

📊 Final Dataset Summary:
   • Total samples: 509,006
   • Compounds: 10,107
   • Cell lines: 5